In [1]:
# Bonus Exercise
# This exercise is an extension to Exercise 2. Build a job that output top 5 causes of death for 
# each gender and age group of all history which updates the result whenever you feed in new data. 

# Your demo should be like the following. You start the job. After feeding the 2005 data to it, 
# it outputs the answer to Exercise 2 for the year 2005. Then feed the 2006 data to it, it outputs 
# the top 5 causes of death for each gender and age group in 2005 and 2006. And so, on

In [1]:
import pandas as pd
import numpy as np
import json

In [39]:
def get_top_causes(df, year):
    # Load json file with code names into a dataframe
    with open('mortality/2015_codes.json') as f: 
        codes_json = json.load(f)
    codes = pd.DataFrame(codes_json)
    
    # Get human-readable names for causes of death
    causes = codes[['39_cause_recode']].dropna().reset_index()
    causes['index'] = causes['index'].astype(int)
        
    # Get human-readable names for age groups
    ages = codes[['age_recode_12']].dropna().reset_index()
    ages['index'] = ages['index'].astype(int)
    
    # Match death records to readable cause names and age groups
    # Drop redundant columns

    deaths_merged = deaths.merge(causes, how='left', left_on='39_cause_recode', right_on='index').drop(['39_cause_recode_x', 'index'], axis=1)
    deaths_merged = deaths_merged.merge(ages, how='left', left_on='age_recode_12', right_on='index').drop(['age_recode_12_x', 'index'], axis=1) 

    # Group death records by gender + by age and sort by count of death causes
    # Columns could use some renaming

    deaths_by_sex = deaths_merged.groupby(['sex', '39_cause_recode_y', 'year']).count().reset_index().sort_values(['sex', 'age_recode_12_y'], ascending=False)
    deaths_by_age = deaths_merged.groupby(['age_recode_12_y', '39_cause_recode_y', 'year']).count().reset_index().sort_values(['age_recode_12_y', 'sex'], ascending=False)
    
    # Get top 5 causes of death for men
    m_deaths = deaths_by_sex[deaths_by_sex['sex'] == 'M'].head(5).drop(['age_recode_12_y'], axis=1)
    m_deaths['Variable'] = 'Sex'
    m_deaths['Rank'] = m_deaths.reset_index().index + 1
    m_deaths.columns = ['Value', 'Top 5 Death Causes', 'Year', 'Variable', 'Rank']

    # Get top 5 causes of death for women
    f_deaths = deaths_by_sex[deaths_by_sex['sex'] == 'F'].head(5).drop(['age_recode_12_y'], axis=1)
    f_deaths['Variable'] = 'Sex'
    f_deaths['Rank'] = f_deaths.reset_index().index + 1
    f_deaths.columns = ['Value', 'Top 5 Death Causes', 'Year', 'Variable', 'Rank']

    output = pd.concat([m_deaths, f_deaths])

    # Get top 5 causes of death for all age groups
    for index,age in ages.iterrows():
        age_deaths = deaths_by_age[deaths_by_age['age_recode_12_y'] == age['age_recode_12']].head(5).drop(['sex'], axis=1)
        age_deaths['Variable'] = 'Age group'
        age_deaths['Rank'] = age_deaths.reset_index().index + 1
        age_deaths.columns = ['Value', 'Top 5 Death Causes', 'Year', 'Variable', 'Rank']
        output = pd.concat([output, age_deaths])

    output = output.set_index(['Variable', 'Value', 'Top 5 Death Causes', 'Year'])
    return output    
    
    

In [41]:
outputs = pd.DataFrame()

for year in range(2005, 2016):
    deaths = pd.read_csv('mortality/' + str(year) + '_data.csv', usecols=["sex", "age_recode_12", "39_cause_recode"])
    deaths['year'] = year
    outputs = pd.concat([outputs, get_top_causes(deaths, year)])
    outputs.to_csv('bonus_outputs/output_2005_to_' + str(year) + '.csv')
    print('2005 to ' + str(year) + ' successful')

2005 to 2005 successful
2005 to 2006 successful
2005 to 2007 successful
2005 to 2008 successful
2005 to 2009 successful
2005 to 2010 successful
2005 to 2011 successful
2005 to 2012 successful
2005 to 2013 successful
2005 to 2014 successful
2005 to 2015 successful
